<h4>Today I'm studying about Word2Vce.</h4> <p></p>
<h4>I'm using tensorflow as a simple example</h4>

**First I'm setting some corpus to create word vectors**

In [33]:
corpus = ['king is a strong man', 
          'queen is a wise woman', 
          'boy is a young man',
          'girl is a young woman',
          'prince is a young king',
          'princess is a young queen',
          'man is strong', 
          'woman is pretty',
          'prince is a boy will be king',
          'princess is a girl will be queen']

**Remove Stop words**<p></p>
In order for efficiency of creating word vector, we will remove commonly used words

In [34]:
def remove_stop_words(corpus):
    stop_words = ['is', 'a', 'will', 'be']
    results = []
    for text in corpus:
        tmp = text.split(' ')
        for stop_word in stop_words:
            if stop_word in tmp:
                tmp.remove(stop_word)
        results.append(' '.join(tmp))
    
    return results

In [35]:
corpus = remove_stop_words(corpus)
print(corpus)

In [36]:
words = []
for text in corpus:
    for word in text.split(' '):
        words.append(word)

words = set(words)
words

<h3>Data Generation</h3>
- we will generate label for each word using skip gram

In [37]:
word_to_id = {}
for i, word in enumerate(words):
    word_to_id[word] = i

sentences = []
for sentence in corpus:
    sentences.append(sentence.split())

window_size = 2

data = []
for sentence in sentences:
    for idx, word in enumerate(sentence):
        for neighbor in sentence[max(idx - window_size, 0) : min(idx + window_size, len(sentence))+1]:
            if neighbor != word:
                data.append([word, neighbor])        

In [38]:
print(data)

In [39]:
word_to_id

In [40]:
import pandas as pd
# for text in corpus:
#     print(text)

df = pd.DataFrame(data, columns = ['input', 'label'])
print(df)

In [41]:
df.shape

In [42]:
word_to_id

## Define Tensorflow Graph

In [43]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

ONE_HOT_DIM = len(words)

def to_one_hot_encoding(data_point_index):
    one_hot_encoding = np.zeros(ONE_HOT_DIM)
    one_hot_encoding[data_point_index] = 1
    return one_hot_encoding

X = []
Y = []

for x, y in zip(df['input'], df['label']):
    X.append(to_one_hot_encoding(word_to_id[x]))
    Y.append(to_one_hot_encoding(word_to_id[y]))    

X_train = np.asarray(X)
Y_train = np.asarray(Y)

x = tf.placeholder(tf.float32, shape = (None, ONE_HOT_DIM))
y_label = tf.placeholder(tf.float32, shape = (None, ONE_HOT_DIM))
EMBEDDING_DIM = 2

w1 = tf.Variable(tf.random_normal([ONE_HOT_DIM, EMBEDDING_DIM]))
b1 = tf.Variable(tf.random_normal([1]))
hidden_layer = tf.add(tf.matmul(x, w1), b1)

w2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, ONE_HOT_DIM]))
b2 = tf.Variable(tf.random_normal([1]))
prediction = tf.nn.softmax(tf.add(tf.matmul(hidden_layer, w2), b2))

loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(prediction), axis = [1]))

train_op = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

In [44]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

iteration = 20000
for i in range(iteration):
    sess.run(train_op, feed_dict = {x : X_train, y_label : Y_train})
    if i % 3000 == 0:
        print('iteration ' + str(i) + 'loss is : ', sess.run(loss, feed_dict = {x : X_train, y_label : Y_train}))

In [45]:
vectors = sess.run(w1 + b1)
print(vectors)

In [46]:
w2v_df = pd.DataFrame(vectors, columns = ['x1', 'x2'])
w2v_df['word'] = list(words)
w2v_df = w2v_df[['word', 'x1', 'x2']]
w2v_df

In [47]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()

for word, x1, x2 in zip(w2v_df['word'], w2v_df['x1'], w2v_df['x2']):
    ax.annotate(word, (x1, x2))

PADDING = 1.0

x_axis_min = np.amin(vectors, axis=0)[0] - PADDING
y_axis_min = np.amin(vectors, axis=0)[1] - PADDING
x_axis_max = np.amax(vectors, axis=0)[0] + PADDING
y_axis_max = np.amax(vectors, axis=0)[1] + PADDING

plt.xlim(x_axis_min, x_axis_max)
plt.ylim(y_axis_min, y_axis_max)
plt.rcParams['figure.figsize'] = (10, 10)

plt.show()